In [1]:
from cataclop.core.models import *

In [2]:
from cataclop.pmu.models import *

In [20]:
races = Race.objects.filter(start_at__date="2018-10-01")

for race in races:
    print(race, race.declared_player_count, race.player_set.count())

2018-10-01 R1C1 13:10 12 12
2018-10-01 R1C2 13:47 16 16
2018-10-01 R1C3 14:22 16 16
2018-10-01 R1C4 14:50 13 13
2018-10-01 R1C5 15:50 12 12
2018-10-01 R1C6 16:23 12 12
2018-10-01 R1C7 16:55 9 9
2018-10-01 R1C8 17:25 16 16
2018-10-01 R1C9 17:55 16 16
2018-10-01 R2C1 11:07 7 7
2018-10-01 R2C2 11:35 8 8
2018-10-01 R2C3 12:08 9 9
2018-10-01 R2C4 12:40 8 8
2018-10-01 R2C5 13:10 10 10
2018-10-01 R3C1 10:50 5 5
2018-10-01 R3C2 11:23 5 5
2018-10-01 R3C3 11:56 6 6
2018-10-01 R3C4 12:23 7 7
2018-10-01 R3C5 12:55 9 9
2018-10-01 R3C6 13:25 9 9
2018-10-01 R3C7 14:05 7 7
2018-10-01 R3C8 14:35 9 9
2018-10-01 R4C1 15:07 14 14
2018-10-01 R4C2 15:37 12 12
2018-10-01 R4C3 16:07 14 14
2018-10-01 R4C4 16:40 7 7
2018-10-01 R4C5 17:10 13 13
2018-10-01 R4C6 17:40 7 7
2018-10-01 R4C7 18:10 5 5
2018-10-01 R4C8 18:40 6 6
2018-10-01 R4C9 19:10 8 8
2018-10-01 R5C1 18:25 16 16
2018-10-01 R5C2 18:55 16 16
2018-10-01 R5C3 19:25 15 15
2018-10-01 R5C4 19:50 11 11
2018-10-01 R5C5 20:15 16 16
2018-10-01 R5C6 20:40 14 14


In [11]:
players = Player.objects.filter(final_odds_ref_offline__isnull=False, position=1)

for p in players:
    print(p.race, p.position, p.final_odds_ref, p.final_odds_ref_offline)

2019-02-01 R1C4 15:32 1 11.9 14.5
2019-02-01 R1C6 16:40 1 6.4 5.3
2019-02-01 R2C5 14:04 1 15.2 21.5
2019-02-01 R5C1 18:27 1 1.2 1.1
2019-02-01 R5C2 18:57 1 12.7 1.9
2019-02-01 R5C3 19:27 1 1.3 1.1
2019-02-01 R5C4 19:57 1 1.4 1.1


In [ ]:
bets = Bet.objects.filter(player__isnull=False).order_by('-created_at')

for bet in bets:
    print( '{} {} / {}: bet {}€ on {}'.format(bet.created_at, bet.url, bet.program, bet.amount, bet.player.num))

In [ ]:
horse = Horse.objects.filter(name__icontains='what secret').first()
horse

In [ ]:
player = Player.objects.filter(horse=horse, num=3).last()
player.race.id

In [ ]:
odds = Odds.objects.filter(player=player)
odds

In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from cataclop.ml import preprocessing
from cataclop.ml import exploration

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

from cataclop.ml.pipeline import factories

In [ ]:
dataset = factories.Dataset.factory('2019-01-07', params={
    'race_id':60962
})

In [ ]:
dataset.load()

In [ ]:
dataset.players[['num', 'horse_id', 'final_odds_ref', 'position']]